# 모델 배포

실행된 파이프라인은 지정된 모델 패키지 그룹 내에 모델 패키지 버전을 생성합니다. 특히 모델의 등록/모델 패키지 생성은 `PendingManualApproval(수동 승인 대기)` 상태로 승인되었습니다.

세이지메이커 파이프라인의 일환으로 데이터 과학자들은 CI/CD 워크플로우의 일부로 승인되었거나 수동 승인 대기 상태로 모델을 등록할 수 있습니다.

또한 아래에 표시된 대로 세이지메이커 스튜디오 UI를 사용하거나 프로그래밍 방식으로 모델을 승인할 수 있습니다.

In [ ]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='gaia/1.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)

# 모델 엔드포인트 가져오기

In [ ]:
%store -r pipeline_endpoint_name

In [ ]:
try:
    pipeline_endpoint_name
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run previous notebooks before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(pipeline_endpoint_name)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b><a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">세이지메이커 HTTPS 엔드포인트</a> 검토하기</b>'.format(
            region, pipeline_endpoint_name
        )
    )
)

# _엔드포인트가 배포될 때까지 기다리기_
_참고: 이 과정은 몇 분 정도 소요됩니다. 잠시 기다려 주세요._

In [ ]:
%%time

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=pipeline_endpoint_name)

# _위의 엔드포인트가 배포될 때까지 기다리기_

# 제로샷 추론

In [ ]:
import json
from sagemaker import Predictor

zero_shot_prompt = """Summarize the following conversation.

#Person1#: Tom, I've got good news for you.
#Person2#: What is it?
#Person1#: Haven't you heard that your novel has won The Nobel Prize?
#Person2#: Really? I can't believe it. It's like a dream come true. I never expected that I would win The Nobel Prize!
#Person1#: You did a good job. I'm extremely proud of you.
#Person2#: Thanks for the compliment.
#Person1#: You certainly deserve it. Let's celebrate!

Summary:"""
predictor = Predictor(
    endpoint_name=pipeline_endpoint_name,
    sagemaker_session=sess,
)
response = predictor.predict(zero_shot_prompt,
        {
            "ContentType": "application/x-text",
            "Accept": "application/json",
        },
)
response_json = json.loads(response.decode('utf-8'))
print(response_json)

# 정리: 엔드포인트 종료

In [ ]:
# sm.delete_endpoint(
#     EndpointName=pipeline_endpoint_name
# )